# С помощью данной методики можно найти свободные IMSI диапазоны, а также создать файл с ключами для отправки кастомеру

In [1]:
# Загрузить данные из файлов
# Задать имена аккаунтов и файлов
#----------------------------------

import pandas as pd
from pandas import DataFrame
from os.path import join, normpath
import datetime as dt
from collections import namedtuple

### Set the variables here ###

reseller_name = 'Roamability Resellers Test'
customer_name = 'Tcom' # to use as a file name

account_s1 = 'Tcom' # Account name with S1
account_s2 = 'Tcom' # Account name with S2
account_s4 = '' # Account name with S4
account_s5 = '' # Account name with S5
account_s6 = '' # Account name with S6
account_s8 = '' # Account name with S8

##############################

sponsor_object = namedtuple('SponsorRange', 'sponsor test_range prod_range account_name')

s1_sponsor = sponsor_object('S1', '425019613998', '42501', account_s1)
s2_sponsor = sponsor_object('S2', '260060149991', '26006', account_s2)
s4_sponsor = sponsor_object('S4', '454030227305', '45403', account_s4)
s5_sponsor = sponsor_object('S5', '515030191000', '51503', account_s5)
s6_sponsor = sponsor_object('S6', '260036610100', '26003', account_s6)
s8_sponsor = sponsor_object('S8', '234500026400', '23450', account_s8)

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\SoftProfiles\Data'
result_file = 'result_file.csv'
files = ['rb_42501_25_06_20181.txt',
         'rb_42501_2_09_20181.txt',
         'rb_26006_25_06_20181.txt',
         'rb_26006_2_09_20181.txt',
         'rb_26006_30_04_2019.txt',
         'rb_42501_30_04_2019.txt',
         'rb_45403_09_07_2019.txt',
         '454030227306000_454030227308999.out',
         '454030227310000_454030227310999.out',
         '515030191000000_515030191099999.out',
         'rb_soft_191103_s1.txt',
         'rb_soft_191103_s2.txt',
         'rb_soft_191103_s4.txt',
         'rb_soft_191107_s6.txt',
         'rb_soft_191107_s8.txt',
         'rb_soft_191207_s1.txt',
         'rb_soft_191207_s2.txt',]

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor]

current_date = dt.datetime.now().strftime('%y%m%d')

df_imsi = DataFrame(columns = ['IMSI', 'KI', 'OPC'])

for file in files:
    df_imsi = df_imsi.append(pd.read_csv(join(downloads, file), sep=' ', engine='python'))

df_imsi.IMSI = df_imsi.IMSI.astype('int64')
df_imsi.sort_values(by = 'IMSI', inplace = True)

df_imsi.head(3)

,IMSI,KI,OPC
0,234500026400000,693E4A067EEB7F66AD9FBD5D0AA3D1E6,79EC10C3C95CED20C758681507904E6D
1,234500026400001,0C12B4359E0AF6FA8D25E39C0662C3D5,02DECFCAE2F9A6D1AED425ED810B2CA6
2,234500026400002,EC3655E2752449C2E8B1F96F79F9A2FD,79798D3604C1EDF288F237963111E7FB


In [7]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND
(si.IMSI BETWEEN '260060140890000' AND '260060140899999'
OR si.IMSI BETWEEN '260060149991100' AND '260060149999999'
OR si.IMSI BETWEEN '425019613998060' AND '425019613999999'
OR si.IMSI BETWEEN '425019629920000' AND '425019629939999'
OR si.IMSI BETWEEN '425019613790000' AND '425019613799999'
OR si.IMSI BETWEEN '425019613788000' AND '425019613789999'
OR si.IMSI BETWEEN '260060149982000' AND '260060149989999'
OR si.IMSI BETWEEN '454030227305000' AND '454030227305999'
OR si.IMSI BETWEEN '454030227306000' AND '454030227308999'
OR si.IMSI BETWEEN '454030227310000' AND '454030227310999'
OR si.IMSI BETWEEN '515030191000000' AND '515030191009999'
OR si.IMSI BETWEEN '425019613960000' AND '425019613989999'
OR si.IMSI BETWEEN '260060149940000' AND '260060149969999'
OR si.IMSI BETWEEN '454030227311000' AND '454030227319999'
OR si.IMSI BETWEEN '260036610100000' AND '260036610100999'
OR si.IMSI BETWEEN '234500026400000' AND '234500026400999'
OR si.IMSI BETWEEN '425019613930000' AND '425019613959999'
OR si.IMSI BETWEEN '260060149910000' AND '260060149939999')
'''

# OR si.IMSI BETWEEN '515030191000000' AND '515030191099999' # Full range of Smart

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
0,Clementvale,Clementvale S1 profiles,Active,425019613791001
1,Clementvale,Clementvale S1 profiles,Active,425019613791008
2,Clementvale,Clementvale S1 profiles,Active,425019613791010


In [8]:
# Объединяем IMSI с ключами и данные OCS
#----------------------------------

df_ocs_imsi_ki = pd.merge(df_imsi, df_ocs_imsi, how='left', on='IMSI')
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_ocs_imsi_ki.IMSI.astype('str').str.slice(0,12)]
df_ocs_imsi_ki_group = df_ocs_imsi_ki.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')

df_ocs_imsi_ki_group.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
205,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400002,234500026400999,998
233,Roamability Test RT,Sigos Test Sim cards,Active,234500026400,234500026400000,234500026400000,1
204,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610100,260036610100001,260036610100999,999


In [9]:
# For test accounts to use the following ranges:
# S1: 425019613998*
# S2: 260060149991*
# S4: 4540302273050*
# S5: 515030191000*
# Выбрать свободные из "Roamability Invertory"

for sponsor in sponsors:
    print(f'{sponsor.sponsor} test IMSIs in OCS by Reseller:')
    display(df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['IMSI'] == sponsor.test_range].sort_values('IMSI_MIN'))

S1 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
21,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,425019613998,425019613998060,425019613998064,5
121,MCN Telecom,MCN Telecom Soft SIM,Active,425019613998,425019613998065,425019613998067,3
244,Tottolli-Tech-Mobil,Test Sim cards,Active,425019613998,425019613998071,425019613998081,4
10,Dialoq - Genesisclub RT,Genesisclub Partner Soft test,Active,425019613998,425019613998072,425019613998937,18
109,Joy_telecom,Joy telecom Partner Soft test,Active,425019613998,425019613998088,425019613998929,5
274,Watchdata SG,Watchdata Partner Soft test,Active,425019613998,425019613998090,425019613998090,1
148,RedTea RT,RedTea TEST Soft-simcards account,Active,425019613998,425019613998093,425019613998491,22
219,Roamability Resellers Test,Clementvale S1 soft,Active,425019613998,425019613998113,425019613998990,6
216,Roamability Resellers Test,1OT S1 Soft,Active,425019613998,425019613998118,425019613998119,2
4,DENT,S1Profiles,Active,425019613998,425019613998120,425019613998142,12


S2 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
20,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,260060149991,260060149991100,260060149991104,5
120,MCN Telecom,MCN Telecom Soft SIM,Active,260060149991,260060149991105,260060149991107,3
9,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991,260060149991110,260060149991945,18
275,Watchdata SG,Watchdata P4 Soft test,Active,260060149991,260060149991123,260060149991123,1
17,Drimsim,Drimsim S2 Soft test IMSI,Active,260060149991,260060149991124,260060149991949,12
217,Roamability Resellers Test,1OT S2 Soft,Active,260060149991,260060149991134,260060149991135,2
5,DENT,S2Profiles,Active,260060149991,260060149991136,260060149991158,12
2,DENT,MultiImsiProfiles,Active,260060149991,260060149991138,260060149991478,21
226,Roamability Resellers Test,PointDume Multi,Active,260060149991,260060149991479,260060149991498,20
134,MCN Telecom,MCN Telecom Soft SIM S2,Active,260060149991,260060149991499,260060149991954,10


S4 test IMSIs in OCS by Reseller:


...

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
19,Drimsim,Drimsim S5 Soft test IMSI,Active,515030191000,515030191000000,515030191000024,25
112,Joy_telecom,Joy telecom S5 Soft,Active,515030191000,515030191000025,515030191000034,10
232,Roamability Test RT,RedTea S5 Test SIM cards,Active,515030191000,515030191000035,515030191000035,1
237,Roamability Test RT,Sigos Test Sim cards,Active,515030191000,515030191000036,515030191000036,1
135,Mondicon SIM RT,S5 soft profiles,Active,515030191000,515030191000037,515030191000056,20
206,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191000,515030191000057,515030191000999,943


S6 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
234,Roamability Test RT,Sigos Test Sim cards,Active,260036610100,260036610100000,260036610100000,1
204,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610100,260036610100001,260036610100999,999


S8 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
233,Roamability Test RT,Sigos Test Sim cards,Active,234500026400,234500026400000,234500026400000,1
205,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400002,234500026400999,998


In [5]:
# For commersial usage

for sponsor in sponsors:
    print(f'{sponsor.sponsor} prod IMSIs in OCS by Reseller:')
    display(df_ocs_imsi_ki_group[(df_ocs_imsi_ki_group.IMSI.str.startswith(sponsor.prod_range, na=False))
                                 & (df_ocs_imsi_ki_group.RESELLER_NAME == 'Roamability Invertory')
                                ].sort_values('IMSI_MIN'))

S1 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
182,Roamability Invertory,Partner Soft simcards inventory,Active,425019613955,425019613955000,425019613955999,1000
183,Roamability Invertory,Partner Soft simcards inventory,Active,425019613956,425019613956000,425019613956999,1000
184,Roamability Invertory,Partner Soft simcards inventory,Active,425019613957,425019613957000,425019613957999,1000
185,Roamability Invertory,Partner Soft simcards inventory,Active,425019613958,425019613958000,425019613958999,1000
186,Roamability Invertory,Partner Soft simcards inventory,Active,425019613959,425019613959000,425019613959999,1000
187,Roamability Invertory,Partner Soft simcards inventory,Active,425019613975,425019613975000,425019613975999,1000
188,Roamability Invertory,Partner Soft simcards inventory,Active,425019613976,425019613976000,425019613976999,1000
189,Roamability Invertory,Partner Soft simcards inventory,Active,425019613977,425019613977000,425019613977999,1000
190,Roamability Invertory,Partner Soft simcards inventory,Active,425019613978,425019613978000,425019613978999,1000
191,Roamability Invertory,Partner Soft simcards inventory,Active,425019613979,425019613979000,425019613979999,1000


S2 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
152,Roamability Invertory,P4 Soft simcards inventory,Active,260060149935,260060149935000,260060149935999,1000
153,Roamability Invertory,P4 Soft simcards inventory,Active,260060149936,260060149936000,260060149936999,1000
154,Roamability Invertory,P4 Soft simcards inventory,Active,260060149937,260060149937000,260060149937999,1000
155,Roamability Invertory,P4 Soft simcards inventory,Active,260060149938,260060149938000,260060149938999,1000
156,Roamability Invertory,P4 Soft simcards inventory,Active,260060149939,260060149939000,260060149939999,1000
157,Roamability Invertory,P4 Soft simcards inventory,Active,260060149956,260060149956500,260060149956999,500
158,Roamability Invertory,P4 Soft simcards inventory,Active,260060149957,260060149957000,260060149957999,1000
159,Roamability Invertory,P4 Soft simcards inventory,Active,260060149958,260060149958000,260060149958999,1000
160,Roamability Invertory,P4 Soft simcards inventory,Active,260060149959,260060149959000,260060149959999,1000
161,Roamability Invertory,P4 Soft simcards inventory,Active,260060149960,260060149960000,260060149960999,1000


S4 prod IMSIs in OCS by Reseller:


...

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
206,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191000,515030191000057,515030191000999,943
207,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191001,515030191001300,515030191001999,700
208,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191002,515030191002000,515030191002999,1000
209,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191003,515030191003000,515030191003999,1000
210,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191004,515030191004000,515030191004999,1000
211,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191005,515030191005000,515030191005999,1000
212,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191006,515030191006000,515030191006999,1000
213,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191007,515030191007000,515030191007999,1000
214,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191008,515030191008000,515030191008999,1000
215,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191009,515030191009000,515030191009999,1000


S6 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
204,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610100,260036610100001,260036610100999,999


S8 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
205,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400002,234500026400999,998


In [10]:
# Записать выбранные диапазоны IMSI
# Проверить выбранные IMSI на принадлежность к "Roamability Invertory"

# После переноса IMSI в аккаунты Кастомеров проверить выбранные IMSI и диапазоны
# (предварительно перезагрузив данные из OCS)

imsi_range_object = namedtuple('ImsiRange', 'sponsor imsi_list')

def imsi_list(range_start, range_end):
    return [str(imsi) for imsi in range(range_start, range_end+1,1)]

### Set the IMSI ranges here ###

s1_imsi_range = imsi_range_object('S1', imsi_list(425019613998492, 425019613998492+8))
s2_imsi_range = imsi_range_object('S2', imsi_list(260060149991508, 260060149991508+8))
s4_imsi_range = imsi_range_object('S4', imsi_list(0, 0))
s5_imsi_range = imsi_range_object('S5', imsi_list(0, 0))
s6_imsi_range = imsi_range_object('S6', imsi_list(0, 0))
s8_imsi_range = imsi_range_object('S8', imsi_list(0, 0))

#################################

imsi_ranges = [s1_imsi_range, s2_imsi_range, s4_imsi_range, s5_imsi_range, s6_imsi_range, s8_imsi_range]

for imsi_range in imsi_ranges:
    if imsi_range.imsi_list:
        display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(imsi_range.imsi_list)].groupby(['RESELLER_NAME',
                                                                                'ACCOUNT_NAME','SUB_STATUS']).count())

,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Roamability Resellers Test,Tcom,Active,9


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Roamability Resellers Test,Tcom,Active,9


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


In [11]:
# Выгружаем файлы с ключами для диапазонов IMSI
#----------------------------------

for sponsor, imsi_range in zip(sponsors, imsi_ranges):
    if sponsor.account_name and imsi_range.imsi_list:
        df_ocs_imsi_ki.loc[(df_ocs_imsi_ki.RESELLER_NAME == reseller_name)
                           & (df_ocs_imsi_ki.ACCOUNT_NAME == sponsor.account_name)
                           & (df_ocs_imsi_ki.IMSI.astype('str').isin(imsi_range.imsi_list)),
                           ['IMSI', 'KI', 'OPC']].\
        to_csv(join(downloads, f'{sponsor.sponsor}_{customer_name}_{current_date}.txt'), index=False)
        print(f'{sponsor.sponsor}_{customer_name}_{current_date}.txt')

S1_Tcom_200104.txt
S2_Tcom_200104.txt


### Далее шифруем и отправляем
----------------------------------
1. Упаковать оба файла в zip архив.
2. Выполнить экспорт открытого ключа Кастомера (дважды кликнуть по файлу ключа).
3. В папке с архивом по пр.кл. на zip архив и выбрать "Зашифровать"
4. Зашифровать своим ключем *_FOR_KEYS
5. Отправить кастомеру. Свой ключ отправлять не нужно.